In [51]:
#SDFA-Structure-Aware-Discriminative-Feature-Aggregation-for-Efficient-Human-Fall-Detection-in-Video

##Utils

In [2]:
def import_class(name):
    components = name.split('.')
    mod = __import__(components[0])
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod


def count_params(model):
    #return sum(p.numel() for p in model.parameters() if p.requires_grad)
    return sum(p.numel() for name, p in model.named_parameters() if p.requires_grad and 'fc' not in name)

##Activation

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
#from model.activations import *
def activation_factory(name, inplace=True):
    if name == 'relu':
        return nn.ReLU(inplace=inplace)
    elif name == 'leakyrelu':
        return nn.LeakyReLU(0.2, inplace=inplace)
    elif name == 'tanh':
        return nn.Tanh()
    elif name == 'swish':
        return Swish()
    elif name == 'hardswish':
        return HardSwish()
    elif name == 'metaacon':
        return MetaAconC()
    elif name == 'acon':
        return AconC()
    elif name == 'linear' or name is None:
        return nn.Identity()
    else:
        raise ValueError('Not supported activation:', name)

##Random Drops

In [4]:
import torch
import torch.nn.functional as F
from torch import nn
import warnings
class Randomized_DropBlock_Ske(nn.Module):
    def __init__(self, block_size=7):
        super(Randomized_DropBlock_Ske, self).__init__()
        self.keep_prob = 0.0
        self.block_size = block_size

    def forward(self, input, keep_prob, A, num_point):  # n,c,t,v
        self.keep_prob = keep_prob
        self.num_point = num_point
        if not self.training or self.keep_prob == 1:
            return input
        n, c, t, v = input.size()
        #print(input.shape) 
        input_abs = torch.mean(torch.mean(
            torch.abs(input), dim=2), dim=1).detach()
        input_abs = input_abs / torch.sum(input_abs) * input_abs.numel()
        if self.num_point == 25:  # Kinect V2 
            gamma = (1. - self.keep_prob) / (1 + 1.92)
        elif self.num_point == 20:  # Kinect V1
            gamma = (1. - self.keep_prob) / (1 + 1.9)
        else:
            gamma = (1. - self.keep_prob) / (1 + 1.92)
            warnings.warn('undefined skeleton graph')
        M_seed = torch.bernoulli(torch.clamp(
            input_abs * gamma, max=1.0)).to(device=input.device, dtype=input.dtype)
        #print(M_seed.shape)
        print(A.shape)
        M = torch.matmul(M_seed, A)
        #M = torch.einsum('nv,cvw->nv', (M_seed, A)).contiguous()
        M[M > 0.001] = 1.0
        M[M < 0.5] = 0.0
        #print(M.shape)
        mask = (1 - M).view(n, 1, 1, self.num_point)
        #print(M.shape)
        return input * mask * mask.numel() / mask.sum()
    
    
class Randomized_DropBlockT_1d(nn.Module):
    def __init__(self, block_size=7):
        super(Randomized_DropBlockT_1d, self).__init__()
        self.keep_prob = 0.0
        self.block_size = block_size

    def forward(self, input, keep_prob):
        self.keep_prob = keep_prob
        if not self.training or self.keep_prob == 1:
            return input
        n,c,t,v = input.size()

        input_abs = torch.mean(torch.mean(torch.abs(input),dim=3),dim=1).detach()
        input_abs = (input_abs/torch.sum(input_abs)*input_abs.numel()).view(n,1,t)
        gamma = (1. - self.keep_prob) / self.block_size
        input1 = input.permute(0,1,3,2).contiguous().view(n,c*v,t)
        M = torch.bernoulli(torch.clamp(input_abs * gamma, max=1.0)).repeat(1,c*v,1)
        Msum = F.max_pool1d(M, kernel_size=[self.block_size], stride=1, padding=self.block_size // 2)
        idx = torch.randperm(Msum.shape[2])
        RMsum = Msum[:,:,idx].view(Msum.size()) ## shuffles MSum to drop random frames instead of dropping a block of frames
        mask = (1 - RMsum).to(device=input.device, dtype=input.dtype)
        #print(mask.shape)
        return (input1 * mask * mask.numel() /mask.sum()).view(n,c,v,t).permute(0,1,3,2)
    
    
if __name__ == "__main__":
       
    #dropS = DropBlock_Ske()
    x = torch.randn(12,96,50,6)
    #x = torch.randn(12,96,75,6)
    A = torch.randn(3,6,25)

    #out = dropS(x, 0.9, A, x.shape[3])
    dropT = Randomized_DropBlockT_1d(block_size=41)
    #dropT = DropBlockT_1d(block_size=41)
    #mask, out = dropT(x, 0.9)
    mask= dropT(x, 0.9)
    

##Layer

In [38]:
"""
Created on Sun Sep 12 23:53:06 2021
@author: sania
"""
import torch
import torch.nn as nn
#from model.activation import activation_factory
#from model.Random_Drops import *

# Thanks to YAN Sijie for the released code on Github (https://github.com/yysijie/st-gcn)
class SpatialGraphConv(nn.Module):
    def __init__(self, in_channel, out_channel, max_graph_distance, bias, edge, A, act_type, keep_prob, block_size, 
                 num_point, residual=True, **kwargs):
        super(SpatialGraphConv, self).__init__()
        self.keep_prob = keep_prob
        self.num_point = num_point
        self.s_kernel_size = max_graph_distance + 1
        self.gcn = nn.Conv2d(in_channel, out_channel, 1, bias=bias)
        self.A = nn.Parameter(A, requires_grad=False)
        if edge:
            self.edge = nn.Parameter(torch.ones_like(self.A))
        else:
            self.edge = 1
            
        self.act = activation_factory(act_type)   
        self.bn = nn.BatchNorm2d(out_channel)
        
        if residual and in_channel != out_channel:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, 1, bias=bias),
                nn.BatchNorm2d(out_channel),
            )
        self.dropS = Randomized_DropBlock_Ske()
        self.dropT = Randomized_DropBlockT_1d(block_size=block_size)
         
    def forward(self, x):
        res = self.residual(x)
        x = self.gcn(x)
        n, kc, t, v = x.size()
        #x = x.view(n, self.s_kernel_size, kc//self.s_kernel_size, t, v)
        print(self.A.shape)
        print(self.edge.shape)
        #print(x.shape)
        x = torch.einsum('nctv,cvw->nctw', (x, self.A * self.edge)).contiguous()#n=バッチサイズ,1
        #print(self.A * self.edge)
        #x = self.dropS(self.bn(x), self.keep_prob, self.A * self.edge, self.num_point) + self.dropS(res, self.keep_prob, self.A * self.edge, self.num_point)
        x = self.dropT(self.dropS(self.bn(x), self.keep_prob, self.A * self.edge, self.num_point), self.keep_prob) + self.dropT(self.dropS(res, self.keep_prob, self.A * self.edge, self.num_point), self.keep_prob)
        
        return self.act(x)
    
class SepTemporal_Block(nn.Module):
    def __init__(self, channel, temporal_window_size, bias, act_type, edge, A, num_point, keep_prob, block_size, expand_ratio, stride=1, residual=True, **kwargs):
        super(SepTemporal_Block, self).__init__()
        self.keep_prob = keep_prob
        self.num_point = num_point
        padding = (temporal_window_size - 1) // 2
        self.act = activation_factory(act_type)

        if expand_ratio > 0:
            inner_channel = channel * expand_ratio
            self.expand_conv = nn.Sequential(
                nn.Conv2d(channel, inner_channel, 1, bias=bias),
                nn.BatchNorm2d(inner_channel),
            )
        else:
            inner_channel = channel
            self.expand_conv = None

        self.depth_conv = nn.Sequential(
            nn.Conv2d(inner_channel, inner_channel, (temporal_window_size,1), (stride,1), (padding,0), groups=inner_channel, bias=bias),
            nn.BatchNorm2d(inner_channel),
        )
        self.point_conv = nn.Sequential(
            nn.Conv2d(inner_channel, channel, 1, bias=bias),
            nn.BatchNorm2d(channel),
        )
        if not residual:
            self.residual = lambda x:0
        elif stride == 1:
            self.residual = nn.Identity()
        else:
            self.residual = nn.Sequential(
                nn.Conv2d(channel, channel, 1, (stride,1), bias=bias),
                nn.BatchNorm2d(channel),
            )
        self.A = nn.Parameter(A, requires_grad=False)
        if edge:
            self.edge = nn.Parameter(torch.ones_like(self.A))
        else:
            self.edge = 1
        self.dropS = Randomized_DropBlock_Ske()
        self.dropT = Randomized_DropBlockT_1d(block_size=block_size)
        
    def forward(self, x):
        res = self.residual(x)
        if self.expand_conv is not None:
            x = self.act(self.expand_conv(x))
        x = self.act(self.depth_conv(x))
        x = self.point_conv(x)
        #x = self.dropT(x, self.keep_prob) + self.dropT(res, self.keep_prob)
        x = self.dropT(self.dropS(x, self.keep_prob, self.A * self.edge, self.num_point), self.keep_prob) + self.dropT(self.dropS(res, self.keep_prob, self.A * self.edge, self.num_point), self.keep_prob)
        return self.act(x)

##AdjGraph

In [39]:
import numpy as np

class adjGraph():
    """ The Graph to model the skeletons extracted by the openpose
    Args:
        strategy (string): must be one of the follow candidates
        - uniform: Uniform Labeling
        - distance: Distance Partitioning
        - spatial: Spatial Configuration
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        layout (string): must be one of the follow candidates
        - openpose: Is consists of 18 joints. For more information, please
            refer to https://github.com/CMU-Perceptual-Computing-Lab/openpose#output
        - ntu-rgb+d: Is consists of 25 joints. For more information, please
            refer to https://github.com/shahroudy/NTURGB-D
        max_hop (int): the maximal distance between two connected nodes
        dilation (int): controls the spacing between the kernel points
    """

    def __init__(self,
                 layout='ntu-rgb+d',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(
            self.num_node, self.edge, max_hop=max_hop)
        self.get_adjacency(strategy)

    def __str__(self):
        return self.A

    def get_edge(self, layout):
        if layout == 'openpose':
            self.num_node = 18
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(4, 3), (3, 2), (7, 6), (6, 5), (13, 12), (12,
                                                                        11),
                             (10, 9), (9, 8), (11, 5), (8, 2), (5, 1), (2, 1),
                             (0, 1), (15, 0), (14, 0), (17, 15), (16, 14)]
            self.edge = self_link + neighbor_link
            self.center = 1
        elif layout == 'ntu-rgb+d':
            self.num_node = 25
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_1base = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                              (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                              (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                              (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                              (22, 23), (23, 8), (24, 25), (25, 12)]
            neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
            self.edge = self_link + neighbor_link
            self.center = 21 - 1
        elif layout == 'ntu_edge':
            self.num_node = 24
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_1base = [(1, 2), (3, 2), (4, 3), (5, 2), (6, 5), (7, 6),
                              (8, 7), (9, 2), (10, 9), (11, 10), (12, 11),
                              (13, 1), (14, 13), (15, 14), (16, 15), (17, 1),
                              (18, 17), (19, 18), (20, 19), (21, 22), (22, 8),
                              (23, 24), (24, 12)]
            neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
            self.edge = self_link + neighbor_link
            self.center = 2
        # elif layout=='customer settings'
        #     pass
        elif layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
        else:
            raise ValueError("Do Not Exist This Layout.")

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[
                                    i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.
                                              center] > self.hop_dis[i, self.
                                                                     center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
        else:
            raise ValueError("Do Not Exist This Strategy")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

In [40]:
class Graph:
    """The Graph to model the skeletons extracted by the Alpha-Pose.
    Args:
        - strategy: (string) must be one of the follow candidates
            - uniform: Uniform Labeling,
            - distance: Distance Partitioning,
            - spatial: Spatial Configuration,
        For more information, please refer to the section 'Partition Strategies'
            in our paper (https://arxiv.org/abs/1801.07455).
        - layout: (string) must be one of the follow candidates
            - coco_cut: Is COCO format but cut 4 joints (L-R ears, L-R eyes) out.
        - max_hop: (int) the maximal distance between two connected nodes.
        - dilation: (int) controls the spacing between the kernel points.
    """
    def __init__(self,
                 layout='coco_cut',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1):
        self.max_hop = max_hop
        self.dilation = dilation

        self.get_edge(layout)
        self.hop_dis = get_hop_distance(self.num_node, self.edge, max_hop)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'coco_cut':
            self.num_node = 14
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(6, 4), (4, 2), (2, 13), (13, 1), (5, 3), (3, 1), (12, 10),
                             (10, 8), (8, 2), (11, 9), (9, 7), (7, 1), (13, 0)]
            self.edge = self_link + neighbor_link
            self.center = 13
        else:
            raise ValueError('This layout is not supported!')

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis ==
                                                                hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
            #self.A = np.swapaxes(np.swapaxes(A, 0, 1), 1, 2)
        else:
            raise ValueError("This strategy is not supported!")


def get_hop_distance(num_node, edge, max_hop=1):
    A = np.zeros((num_node, num_node))
    for i, j in edge:
        A[j, i] = 1
        A[i, j] = 1

    # compute hop steps
    hop_dis = np.zeros((num_node, num_node)) + np.inf
    transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
    arrive_mat = (np.stack(transfer_mat) > 0)
    for d in range(max_hop, -1, -1):
        hop_dis[arrive_mat[d]] = d
    return hop_dis


def normalize_digraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-1)
    AD = np.dot(A, Dn)
    return AD


def normalize_undigraph(A):
    Dl = np.sum(A, 0)
    num_node = A.shape[0]
    Dn = np.zeros((num_node, num_node))
    for i in range(num_node):
        if Dl[i] > 0:
            Dn[i, i] = Dl[i]**(-0.5)
    DAD = np.dot(np.dot(Dn, A), Dn)
    return DAD

##Model

In [42]:
import sys
sys.path.insert(0, '')

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
#from model.layers import *
#from utils import import_class
#from Utils import Graph

class cnn1x1(nn.Module):
    def __init__(self, dim1 = 3, dim2 =3, bias = True):
        super(cnn1x1, self).__init__()
        self.cnn = nn.Conv2d(dim1, dim2, kernel_size=1, bias=bias)

    def forward(self, x):
        x = self.cnn(x)
        return x
    
class norm_data(nn.Module):
    def __init__(self, dim= 64):
        super(norm_data, self).__init__()

        self.bn = nn.BatchNorm1d(dim* 14) # dim * num_nodes

    def forward(self, x):
        bs, c, num_joints, step = x.size()
        x = x.view(bs, -1, step)
        x = self.bn(x)
        x = x.view(bs, -1, num_joints, step).contiguous()
        return x
    
class embed(nn.Module):
    def __init__(self, dim, dim1, att_type, norm = True, bias = False):
        super(embed, self).__init__()

        if norm:
            self.cnn = nn.Sequential(                        
                norm_data(dim),
                cnn1x1(dim, dim1, bias=bias),
                nn.ReLU()
            )
        else:
            self.cnn = nn.Sequential(
                cnn1x1(dim, dim1, bias=bias),
                nn.ReLU()
            )
        #self.attention =  Attention_Layer(dim1,  att_type=att_type)

    def forward(self, x):
        x = self.cnn(x)
        #print(x.shape)
        return x#self.attention(x)


def init_param(modules):
    for m in modules:
        if isinstance(m, nn.Conv1d) or isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d) or isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm3d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Conv3d) or isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.001)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
                
class Model(nn.Module):
    def __init__(self,
                 num_class,
                 num_point,
                 max_frame,
                 graph,
                 act_type, 
                 bias,
                 edge,
                 block_size):
        super(Model, self).__init__()
        
        self.num_class =  num_class
        temporal_window_size = 3
        max_graph_distance = 2
        keep_prob = 0.9
        #Graph = import_class(graph)
        Graph = graph
        #A_binary = torch.Tensor(Graph().A_binary)
        A_binary = torch.Tensor(Graph.A)
        #A = torch.rand(3,25,25).cuda()#.to(num_class.dtype).to(num_class.device)
        #self.graph_hop = adjGraph(**graph_args)
        #A = torch.tensor(self.graph_hop.A, dtype=torch.float32, requires_grad=False)
        #self.register_buffer('A', A)
        
        # channels
        D_embed = 64
        c1 = D_embed*2
        c2 = c1 * 2     
        #c3 = c2 * 2    
       
        
        
        self.joint_embed = embed(2, D_embed, att_type='stja', norm=True, bias=bias)
        #self.dif_embed = embed(2, D_embed, att_type='stja', norm=True, bias=bias) #601
        #self.attention =  Attention_Layer(D_embed,  max_frame, act_type, att_type='stja')
        
        self.sgcn1 = SpatialGraphConv(D_embed, c1, max_graph_distance, bias, edge, A_binary, act_type, keep_prob, block_size, num_point, residual=True)
        self.tcn11 = SepTemporal_Block(c1, temporal_window_size, bias, act_type, edge, A_binary, num_point, keep_prob, block_size, expand_ratio=0, stride=1, residual=True)
        self.tcn12 = SepTemporal_Block(c1, temporal_window_size+2, bias, act_type, edge, A_binary, num_point, keep_prob, block_size, expand_ratio=0, stride=2, residual=True)
        
        self.sgcn2 = SpatialGraphConv(c1, c2, max_graph_distance, bias, edge, A_binary, act_type, keep_prob, block_size, num_point, residual=True)
        self.tcn21 = SepTemporal_Block(c2, temporal_window_size, bias, act_type, edge, A_binary, num_point, keep_prob, block_size, expand_ratio=0, stride=1, residual=True)
        self.tcn22 = SepTemporal_Block(c2, temporal_window_size+2, bias, act_type, edge, A_binary, num_point, keep_prob, block_size, expand_ratio=0, stride=2, residual=True)
        
        
        self.fc = nn.Linear(c2, num_class)
        #init_param(self.modules())
    
    def forward(self, x):        
        
        N, C, T, V = x.size()
        #dy = x
        
        # Dynamic Representation        
        pos = x.permute(0, 1, 3, 2).contiguous()  # N, C, V, T
        #print(pos.shape):torch.Size([1, 2, 25, 300])
        #dif = pos[:, :, :, 1:] - pos[:, :, :, 0:-1] #  
        #dif = torch.cat([dif.new(N, dif.size(1), V, 1).zero_(), dif], dim=-1)
        
        pos = self.joint_embed(pos)        
        #dif = self.dif_embed(dif)
        dy = pos #+ dif
        #dy = dif
        dy = dy.permute(0,1,3,2).contiguous() # N, C, T, V   
        #print(dy.shape):torch.Size([1, 64, 300, 25])
        #dy = self.attention(dy)
        #dy.register_hook(lambda g: print(g))
      
        #########################
        #out = self.tcn12(self.tcn11(self.sgcn1(dy)))
        #out = self.tcn22(self.tcn21(self.sgcn2(out)))
        #print(out.shape)
        #out_channels = out.size(1)
        #out = out.reshape(N, out_channels, -1)   
        #print(out.shape)
        #out = out.mean(2)
        #print(out.shape)
        #out = self.fc(out)
        
        out = self.sgcn1(dy)
        print(out.size())
        out = self.tcn11(out)
        print(out.size())
        out = self.tcn12(out)
        print(out.size())
        #out = self.tcn12(self.tcn11(self.sgcn1(dy)))
        out = self.sgcn2(out)
        print(out.size())
        out = self.tcn21(out)
        print(out.size())
        out = self.tcn22(out)
        print(out.size())
        #out = self.tcn22(self.tcn21(self.sgcn2(out)))
        #print(out.shape)
        out_channels = out.size(1)
        out = out.reshape(N, out_channels, -1)   
        print(out.shape)
        out = out.mean(2)
        print(out.shape)
        out = self.fc(out)
        print(out.size())
        return out
    
if __name__ == "__main__":
    !pip install thop
    graph=adjGraph(layout='coco_cut',
                 strategy='uniform',)
    #graph=Graph()
    # For debugging purposes
#     cd /home/uniwa/students3/students/22905553/linux/phd_codes/Light_Fall
    import sys
    sys.path.append('..')
    import thop
    from thop import clever_format
     
    model = Model(
        num_class=2,
        num_point=25,
        max_frame=300,
        #graph='graph.ntu_rgb_d.AdjMatrixGraph',
        graph=graph,
        act_type = 'relu',
        bias = True,
        edge = True,
        block_size=41
    )
    #model = model.cuda()
    macs, params = thop.profile(model, inputs=(torch.randn(32,2,300,14),), verbose=False)
    macs, params = clever_format([macs, params], "%.2f")
    #N, C, T, V, M = 6, 3, 300, 25, 2
    
#     x = torch.randn(1, 2, 300, 25)#.cuda()   
#     out = model.forward(x)
    
#     ##
#     # Drop frame
#     import torch.nn.functional as F
#     keep_prob = 0.9
#     block_size = 41
#     input = torch.randn(1,2,300,25)
#     n,c,t,v = input.size()
#     input1 = input.permute(0,1,3,2).contiguous().view(1,c*v,t)
#     input_abs = torch.mean(torch.mean(torch.abs(input),dim=3),dim=1)
#     gamma = (1. - keep_prob) / block_size
#     M = torch.bernoulli(torch.clamp(input_abs * gamma, max=1.0)).repeat(1,c*v,1)
#     Msum = F.max_pool1d(M, kernel_size=[block_size], stride=1, padding=block_size // 2)
    
#     mask = (1 - Msum)
#     drop = (input1 * mask * mask.numel() /mask.sum()).view(n,c,v,t).permute(0,1,3,2)
    
#     idx = torch.randperm(Msum.shape[2])
#     a = Msum[idx].view(Msum.size())
    
#     idx = torch.randperm(Msum.shape[2])
#     a = Msum[:,:,idx].view(Msum.size())
#     mask = (1 - a)
#     drop = (input1 * mask * mask.numel() /mask.sum()).view(n,c,v,t).permute(0,1,3,2)
    
#     # Drop joints
#     from utils import import_class
#     input = torch.randn(1,2,300,25)
#     n,c,t,v = input.size()
#     #Graph = import_class('graph.ntu_rgb_d.AdjMatrixGraph')
#     graph = Graph(**graph_args)
#     A_binary = torch.Tensor(Graph().A_binary)
#     input_abs = torch.mean(torch.mean(torch.abs(input), dim=2), dim=1)
#     input_abs = input_abs / torch.sum(input_abs) * input_abs.numel()
#     gamma = (1. - keep_prob) / (1 + 1.92)
#     M_seed = torch.bernoulli(torch.clamp(input_abs * gamma, max=1.0))
#     M = torch.matmul(M_seed, A_binary)
#     M[M > 0.001] = 1.0
#     M[M < 0.5] = 0.0
#     mask = (1 - M).view(n, 1, 1, 25)    

/bin/bash: pip: command not found
torch.Size([1, 14, 14])
torch.Size([1, 14, 14])
torch.Size([32, 128, 300, 14])
torch.Size([32, 128, 300, 14])
torch.Size([32, 128, 150, 14])
torch.Size([1, 14, 14])
torch.Size([1, 14, 14])
torch.Size([32, 256, 150, 14])
torch.Size([32, 256, 150, 14])
torch.Size([32, 256, 75, 14])
torch.Size([32, 256, 1050])
torch.Size([32, 256])
torch.Size([32, 2])


##train

In [34]:
import os
import time
import torch
import pickle
import numpy as np
import torch.nn.functional as F
from shutil import copyfile
from tqdm import tqdm
from torch.utils import data
from torch.optim.adadelta import Adadelta
from sklearn.model_selection import train_test_split

In [35]:
epochs = 30
batch_size = 32 #32

In [ ]:
data_files = ['../Data/2clCoffee_01_new-set(labelXscrw).pkl',
              '../Data/2clHome_01_new-set(labelXscrw).pkl']

In [ ]:
class_names = ['Fall','No_fall']
num_class = len(class_names)

In [ ]:
def load_dataset(data_files, batch_size, split_size=0.2):#0.2
    """Load data files into torch DataLoader with/without spliting train-test.
    """
    features, labels = [], []
    for fil in data_files:
        with open(fil, 'rb') as f:
            fts, lbs = pickle.load(f)
            features.append(fts)
            labels.append(lbs)
        del fts, lbs
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)

    if split_size > 0:
        x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=split_size,
                                                              random_state=9)
        train_set = data.TensorDataset(torch.tensor(x_train, dtype=torch.float32).permute(0, 3, 1, 2),
                                       torch.tensor(y_train, dtype=torch.float32))
        valid_set = data.TensorDataset(torch.tensor(x_valid, dtype=torch.float32).permute(0, 3, 1, 2),
                                       torch.tensor(y_valid, dtype=torch.float32))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = data.DataLoader(valid_set, batch_size)
    else:
        train_set = data.TensorDataset(torch.tensor(features, dtype=torch.float32).permute(0, 3, 1, 2),
                                       torch.tensor(labels, dtype=torch.float32))
        train_loader = data.DataLoader(train_set, batch_size, shuffle=True)
        valid_loader = None
    return train_loader, valid_loader

def accuracy_batch(y_pred, y_true):
    return (y_pred.argmax(1) == y_true.argmax(1)).mean()


def set_training(model, mode=True):
    for p in model.parameters():
        p.requires_grad = mode
    model.train(mode)
    return model

In [ ]:
if __name__ == '__main__':
    save_folder = os.path.join(os.path.dirname(__file__), save_folder)
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # DATA.
    train_loader, _ = load_dataset(data_files[0:1], batch_size) #batch_size = 32
    
    valid_loader, train_loader_ = load_dataset(data_files[:1], batch_size, 0.2)
    #print("err")
    train_loader = data.DataLoader(data.ConcatDataset([train_loader.dataset, train_loader_.dataset]),
                                   batch_size, shuffle=True)
    dataloader = {'train': train_loader, 'valid': valid_loader}
    del train_loader_
    
    #print(train_loader.shape)
    
    # MODEL.(list化)
    graph_args = {'strategy': 'spatial'}
    model = TwoStreamSpatialTemporalGraph(graph_args, num_class).to(device)
    #model = TwoStreamSpatialTemporalGraph(graph_args, 8).to(device)

    #optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
    #optimizer = Adadelta(model.parameters())
    #optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
    
    
    losser = torch.nn.BCELoss() #fall or no_fall
    #losser = torch.nn.CrossEntropyLoss()

    # TRAINING.
    loss_list = {'train': [], 'valid': []}
    accu_list = {'train': [], 'valid': []}
    for e in range(epochs):
        print('Epoch {}/{}'.format(e, epochs - 1))
        for phase in ['train', 'valid']:
            if phase == 'train':
                model = set_training(model, True)
            else:
                model = set_training(model, False)

            run_loss = 0.0
            run_accu = 0.0
            with tqdm(dataloader[phase], desc=phase) as iterator:
                for pts, lbs in iterator:
                    # Create motion input by distance of points (x, y) of the same node
                    # in two frames.
                    mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]
                    
                    mot = mot.to(device)
                    pts = pts.to(device)
                    lbs = lbs.to(device)
                    #print(pts.size())torch.Size([32, 3, 30, 14])
                    #print(mot.size())torch.Size([32, 2, 29, 14])
                    # Forward.
                    out = model((pts, mot))
                    #print(lbs)

                    #print(out)
                    loss = losser(out, lbs)#エラーが起きた

                    if phase == 'train':
                        # Backward.
                        model.zero_grad()
                        loss.backward()
                        optimizer.step()

                    run_loss += loss.item()
                    accu = accuracy_batch(out.detach().cpu().numpy(),
                                          lbs.detach().cpu().numpy())
                    run_accu += accu

                    iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                        loss.item(), accu))
                    iterator.update()
                    #break
            loss_list[phase].append(run_loss / len(iterator))
            accu_list[phase].append(run_accu / len(iterator))
            #break

        print('Summary epoch:\n - Train loss: {:.4f}, accu: {:.4f}\n - Valid loss:'
              ' {:.4f}, accu: {:.4f}'.format(loss_list['train'][-1], accu_list['train'][-1],
                                             loss_list['valid'][-1], accu_list['valid'][-1]))

        # SAVE.
        torch.save(model.state_dict(), os.path.join(save_folder, 'tsstg-model.pth'))
        '''
        plot_graphs(list(loss_list.values()), list(loss_list.keys()),
                        'Last Train: {:.2f}, Valid: {:.2f}'.format(
                            loss_list['train'][-1], loss_list['valid'][-1]
                        ), 'Loss', xlim=[0, epochs],
                        save=os.path.join(save_folder, 'loss_graph.png'))
        plot_graphs(list(accu_list.values()), list(accu_list.keys()),
                        'Last Train: {:.2f}, Valid: {:.2f}'.format(
                            accu_list['train'][-1], accu_list['valid'][-1]
                        ), 'Accu', xlim=[0, epochs],
                        save=os.path.join(save_folder, 'accu_graph.png'))
        '''
            #break

    del train_loader, valid_loader

    #model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model.pth',map_location=torch.device('cpu'))))
    model.load_state_dict(torch.load(os.path.join(save_folder, 'tsstg-model.pth')))
    # EVALUATION.
    model = set_training(model, False)
    data_file = data_files[1]
    eval_loader, _ = load_dataset([data_file], 32)
    #data_file = data_files[2]
    #eval_loader, _ = load_dataset([data_file], 64)

    print('Evaluation.')
    run_loss = 0.0
    run_accu = 0.0
    y_preds = []
    y_trues = []
    #with tqdm(eval_loader, desc='eval') as iterator:
    #URFD
    with tqdm(dataloader[phase], desc='eval') as iterator:
        for pts, lbs in iterator:
            mot = pts[:, :2, 1:, :] - pts[:, :2, :-1, :]
            mot = mot.to(device)
            pts = pts.to(device)
            lbs = lbs.to(device)

            out = model((pts, mot))
            loss = losser(out, lbs)

            run_loss += loss.item()
            accu = accuracy_batch(out.detach().cpu().numpy(),
                                  lbs.detach().cpu().numpy())
            run_accu += accu

            y_preds.extend(out.argmax(1).detach().cpu().numpy())
            y_trues.extend(lbs.argmax(1).cpu().numpy())

            iterator.set_postfix_str(' loss: {:.4f}, accu: {:.4f}'.format(
                loss.item(), accu))
            iterator.update()

    run_loss = run_loss / len(iterator)
    run_accu = run_accu / len(iterator)

    '''plot_confusion_metrix(y_trues, y_preds, class_names, 'Eval on: {}\nLoss: {:.4f}, Accu{:.4f}'.format(
        os.path.basename(data_file), run_loss, run_accu
    ), 'true', save=os.path.join(save_folder, '{}-confusion_matrix.png'.format(
        os.path.basename(data_file).split('.')[0])))
        '''
    print('Eval Loss: {:.4f}, Accu: {:.4f}'.format(run_loss, run_accu))